In [1]:
import os, os.path
import numpy as np
import pandas as pd
import model_attributes as ma
from attribute_table import AttributeTable
import geo_classes as gc
import setup_analysis as sa
import support_classes as sc
import support_functions as sf
import importlib
import time
import warnings
import matplotlib.pyplot as plt
import geopandas as gpd
import rioxarray as rx
import itertools
import model_afolu as mafl
from typing import *

/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_attributes.py:2635: UserWarning: Invalid subsector attribute 'key_varreqs_partial'. Valid return type values are:'pycategory_primary', 'abv_subsector', 'sector', 'abv_sector', 'key_varreqs_all'
  warnings.warn(f"Invalid subsector attribute '{return_type}'. Valid return type values are:{valid_rts}")
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/model_attributes.py:2392: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for desig, df in df_by_designation:


# Read in GeoTiffs

###   Land Use Cover (LUC) data from Copernicus

- `PROBAV_LC100_global_v3.0.1_{YEAR}-nrt_Discrete-Classification-map_EPSG-4326.tif`
- https://doi.org/10.5281/zenodo.3939049
    - Marcel Buchhorn; Bruno Smets; Luc Bertels; Bert De Roo; Myroslava Lesiv; Nandin-Erdene Tsendbazar; Martin Herold; Steffen Fritz. *Copernicus Global Land Service: Land Cover 100m: collection 3: epoch 2019: Globe.*
    - Resolution: 100m
    - Product User Manual (including categorization): https://zenodo.org/record/4723921


###   World Country Boundaries (ArcGIS/WorldPop)

- `RasterMask_L0_1km.tif`
- https://hub.arcgis.com/documents/WorldPop::national-boundaries/about
    - Lloyd, C. T., H. Chamberlain, D. Kerr, G. Yetman, L. Pistolesi, F. R. Stevens, A. E. Gaughan, J. J. Nieves, G. Hornby, K. MacManus, P. Sinha, M. Bondarenko, A. Sorichetta, and A. J. Tatem, 2019. “Global Spatio-temporally Harmonised Datasets for Producing High-resolution Gridded Population Distribution Datasets”. Big Earth Data (https://doi.org/10.1080/20964471.2019.1625151).
    

In [2]:
dir_data = "/Users/jsyme/Documents/Projects/FY21/SWCHE131_1000/Data/AFOLU"
dir_luc = os.path.join(dir_data, "copernicus_luc")

# set names 
dict_fp_classification = {}
for yr in range(2015, 2020):
    fp_classification = os.path.join(dir_luc, f"PROBAV_LC100_global_v3.0.1_{yr}-nrt_Discrete-Classification-map_EPSG-4326.tif")
    dict_fp_classification.update({yr: fp_classification})
    
fn_countries = "WB_countries_Admin0_10m"
fn_countries_tif = "RasterMask_L0_1km"
fn_cw = "values_info_with_cw_kc_1984_2013.csv"


fp_countries = os.path.join(dir_data, fn_countries, f"{fn_countries}.shp")
fp_countries_tif = os.path.join(dir_data, f"{fn_countries_tif}.tif")
fp_cw = os.path.join(dir_data, fn_cw)

model_afolu = mafl.AFOLU(sa.model_attributes)
regions = sc.Regions(sa.model_attributes)
time_periods = sc.TimePeriods(sa.model_attributes)

/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:1354: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for i, df in df_in_grouped:
/Users/jsyme/Documents/Projects/git_jbus/sisepuede/python/support_functions.py:1354: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for i, df in df_in_grouped:


In [3]:
# convert geotiff to dataframe
rx_array_countries = rx.open_rasterio(fp_countries_tif)
df_countries = rx_array_countries[0].to_pandas()

# convert geotiff to dataframe
rx_array = rx.open_rasterio(dict_fp_classification.get(2019))
df_luc = rx_array[0].to_pandas()


In [43]:
def get_low_res_indices_for_higher_res(
    grid_low: gc.Grid,
    grid_high: gc.Grid,
) -> Union[pd.DataFrame, Tuple[np.array]]:
    """
    Map gridded elements from a higher resolution data frame to a lower resolution 
        dataframe
        
    Function Arguments
    ------------------
    - grid_low: support_classes.Grid (row indexed by latitude, columns are longitude)
        containing gridded data at a lower resolution
    - grid_high: support_classes.Grid (row indexed by latitude, columns are longitude)
        containing gridded data at a higher resolution
    
    Keyword Arguments
    -----------------
    """
    
    ##  INITIALIZATION

    # check orientations
    return_none = (grid_low.orientation_x != grid_high.orientation_x)
    return_none |= (grid_low.orientation_y != grid_low.orientation_y)
    if return_none:
        return None

    
    # get bounds ands starting indices
    dict_bounds_inds = get_shared_bounds_and_indices(grid_high, grid_low)
    x_min, x_max, y_min, y_max = dict_bounds_inds.get("bounds")
    ind_x_high, ind_x_low, ind_y_high, ind_y_low = dict_bounds_inds.get("inds")
    
    
    ##  ITERATE OVER HIGH RESOLUTION INDICES
    
    inds_low_x_by_high_res = np.zeros(len(grid_high.bounds_x))
    inds_low_y_by_high_res = np.zeros(len(grid_high.bounds_y))
    
    # get indices, for each centroid in the high resolution grid, of closes low-res match (x-axis)
    inds_low_x_by_high_res = iterate_high_to_low(
        grid_high.centroids_x,
        grid_low.centroids_x,
        grid_high.delta_x,
        grid_low.delta_x,
        ind_x_high,
        ind_x_low,
        grid_high.orientation_x,
    )
    # for y-axis
    inds_low_y_by_high_res = iterate_high_to_low(
        grid_high.centroids_y,
        grid_low.centroids_y,
        grid_high.delta_y,
        grid_low.delta_y,
        ind_y_high,
        ind_y_low,
        grid_high.orientation_y,
    )
    
    # return indices
    tup_out = inds_low_x_by_high_res, inds_low_y_by_high_res
    
    return tup_out



def get_overlay_bounds(
    grid_1: gc.Grid,
    grid_2: gc.Grid,
) -> Tuple:
    """
    Return bounds to iterate over from two grids
    
        (x_min, x_max, y_min, y_max)
    """
    
    x_max = min(grid_1.x_max, grid_2.x_max)
    x_min = max(grid_1.x_min, grid_2.x_min)

    y_max = min(grid_1.y_max, grid_2.y_max)
    y_min = max(grid_1.y_min, grid_2.y_min)
    
    tup_out = (x_min, x_max, y_min, y_max)

    return tup_out



def get_shared_bounds_and_indices(
    grid_1: gc.Grid,
    grid_2: gc.Grid,
) -> Dict[str, Tuple]:
    """
    For two grids, determine minimal boundaries within the range of
        both grids. Returns a dictionary with tuples:
        
        dict_out["bounds"] = x_min, x_max, y_min, y_max
        dict_out["inds"] = ind_x_1, ind_x_2, ind_y_1, ind_y_2
    """
    # initialize output dictionary
    dict_out = {}
    
    # get overlay bounds and update dict
    bounds = get_overlay_bounds(grid_1, grid_2)
    x_min, x_max, y_min, y_max = bounds
    dict_out.update({"bounds": bounds})
    
    # get starting indices - 1 and 2 will have same orientation
    ind_x_1 = (
        grid_1.get_index_from_bound(x_min, "x")
        if grid_1.orientation_x == "increasing"
        else grid_1.get_index_from_bound(x_max, "x")
    )
    
    ind_x_2 = (
        grid_2.get_index_from_bound(x_min, "x")
        if grid_2.orientation_x == "increasing"
        else grid_2.get_index_from_bound(x_max, "x")
    )
    
    ind_y_1 = (
        grid_1.get_index_from_bound(y_min, "y")
        if grid_1.orientation_y == "increasing"
        else grid_1.get_index_from_bound(y_max, "y")
    )
    
    ind_y_2 = (
        grid_2.get_index_from_bound(y_min, "y")
        if grid_2.orientation_y == "increasing"
        else grid_2.get_index_from_bound(y_max, "y")
    )
    
    tup_inds = (ind_x_1, ind_x_2, ind_y_1, ind_y_2)
    dict_out.update({"inds": tup_inds})
    
    
    return dict_out


    
def iterate_high_to_low(
    centroids_high_res: np.ndarray,
    centroids_low_res: np.ndarray,
    delta_high: float,
    delta_low: float,
    ind_0_high: int,
    ind_0_low: int,
    orientation: str,
) -> np.ndarray:
    """
    Map elements in the lower-resolution grid to the higher-resolution 
        grid. Returns a numpy array with the length of centroids_high_res,
        with each element the index in the associated axis of the lower 
        resolution grid to use.

    Function Arguments
    ------------------
    - centroids_high_res: axis centroids for the higher-resolution grid
    - centroids_low_res: axis centroids for the lower-resolution grid
    - delta_high: grid square width for higher-resolution grid
    - delta_low: grid square width for lower-resolution grid
    - ind_0_high: starting index for the higher-resolution grid
    - ind_0_low: starting index for the low-resolution grid
    - orientation: "increasing" or "decreasing". High and res grid must 
        have same orientation
    """

    inds_low_x_by_high_res = -np.ones(len(centroids_high_res)).astype(int)

    # set sign for delta
    dec_q = (orientation == "decreasing")

    # start by iterating over the 
    i = ind_0_high
    j = ind_0_low

    while (i < len(centroids_high_res)) & (j < len(centroids_low_res)):

        d_to_lr_cur = np.abs(centroids_high_res[i] - centroids_low_res[j])
        add = 1 if (j < len(centroids_low_res) - 1) else 0
        d_to_lr_next = np.abs(centroids_high_res[i] - centroids_low_res[j + add])

        # if closest to current cell in low-res, keep; otherwise, move to next
        j += 0 if (d_to_lr_cur <= d_to_lr_next) else 1

        # assign output
        inds_low_x_by_high_res[i] = j

        i += 1

    return inds_low_x_by_high_res




In [44]:
grid_countries = gc.Grid(df_countries)
grid_luc_2019 = gc.Grid(df_luc)

In [46]:
inds = get_low_res_indices_for_higher_res(
    grid_countries, 
    grid_luc_2019,
)
    
inds

(array([    0,     0,     0, ..., 43199, 43199, 43199]),
 array([  481,   481,   481, ..., 17280, 17280, 17280]))

In [8]:

"""
# generate output data
array_out = -np.ones(grid_high.data.shape)

for i, ind in enumerate(inds_low_y_by_high_res):
    vec = np.array(grid_low.data.iloc[ind])
    vec = vec[inds_low_x_by_high_res]

    array_out[ind] = vec

# 
data_out = pd.DataFrame(array_out)
data_out.set_index = np.array(grid_high.data.index)
data_out.columns = grid_high.data.columns
""";





(array([    0,     0,     0, ..., 43199, 43199, 43199]),
 array([  481,   481,   481, ..., 17280, 17280, 17280]))

In [56]:
class GridFeature:
    """
    Extract a feature
    
    Initialization Arguments
    ------------------------
    - grid: support_classes.Grid object to extract from
    - feature: value to extract
    """
    
    def __init__(self,
        grid: gc.Grid,
        feature: Union[int, float, str],
    ):
        
        self._initialize_feature(grid, feature)
        
        
    
    
    def _initialize_feature(self,
        grid: gc.Grid,
        feature: Union[int, float, str],
    ) -> None:
        """
        Initialize the feature index and some other information. Sets the
            following properties:
            
            self.feature
            self.feature_index
                NOTE: this index is oriented as (x, y), flipped from the numpy 
                array default of (row, col))
        """
        
        # initialize properties
        self.feature = None
        self.feature_index = None
        
        # check
        w = np.where(grid.data == feature)
        if len(w[0]) == 0:
            return None
        
        
        # modify feature specification - 
        self.feature = feature
        self.feature_index = (w[1], w[0])
        
        return None
        
        

In [57]:
gf = GridFeature(grid_low, 12)

In [203]:
w_x, w_y = gf.feature_index


w_x

array([22464, 22368, 22369, ..., 22000, 22001, 22002])

In [233]:
df_inds = pd.DataFrame(
    zip(range(len(inds[0])), inds[0]),
    columns = ["high", "low"]
)
dfg = df_inds.groupby(["low"])


dict_out = {}
for ind_low, df in dfg:
    dict_out.update({ind_low: np.array(df["high"])})

/var/folders/8m/3ll2cn6d1hdcs6gjqxr2jx5d2hffc9/T/ipykernel_35906/3332991278.py:9: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for ind_low, df in dfg:


In [239]:
tmp = [
    dict_out.get(x) for x in w_x
]
tmp = np.concatenate(tmp)

In [242]:
def get_high_res_feature_from_low_value(
    grid_low: gc.Grid,
    grid_high: gc.Grid,
    feature_low: GridFeature,
    inds_low_by_high: Union[Tuple[np.ndarray, np.ndarray], None] = None,
) -> pd.DataFrame:
    """
    Extract elements of grid_high that overlay with the feature_low, an element 
        of grid_low
    
    Function Arguments
    ------------------
    - grid_low: low resolution support_classes.Grid object containing the feature
    - grid_high: high resolution support_classes.Grid object to extract associated
        data from
    - feature_low: feature in grid_low to filter grid_high on
    
    Keyword Arguments
    -----------------
    - inds_low_by_high: optional high-res grid element-wise indices (ordered by axes 
        (x, y)) in low res grid. For exapmle,
        
        (
            np.array([0, 0, 1, 1, 2, 2, 2]),
            np.array([-1, -1, 0, 0, 1, 1])
        )
        
        means that the first two columns of the high-res grid map to the first column
        of the low res grid; the next two map to the second column of the low-res grid,
        and the final three map to the third column of the low res grid. 
        
        Similarly, the first two rows (y axis, second element of tuple) do not overlay
        the low-res grid, while the next two rows correspond with the first row of the 
        low-res grid, and the final two rows of the high-res overlay the first row of
        the low-res grid.
        
        If None, calculates internally.
        
    """
    
    # calculate indices?
    calc_inds_q = not isinstance(inds_low_by_high, Tuple)
    if not calc_inds_q:
        calc_inds_q |= len(inds_low_by_high) != 2
        calc_inds_q |= (
            (not (isinstance(inds_low_by_high[0], np.ndarray) & isinstance(inds_low_by_high[1], np.ndarray)))
            if not calc_inds_q
            else calc_inds_q
        )
        
    if calc_inds_q:
        inds_low_by_high = get_low_res_indices_for_higher_res(
            grid_low, 
            grid_high,
        )
    
    
    # get indices
    inds_low_by_high_x, inds_low_by_high_y = inds_low_by_high
    inds_low_feature_x, inds_low_feature_y = feature_low.feature_index
    

    w_x = np.where(np.in1d(inds_low_by_high_x, inds_low_feature_x))[0]
    w_y = np.where(np.in1d(inds_low_by_high_y, inds_low_feature_y))[0]
    
    return w_x, w_y

    

w_x, w_y = get_high_res_feature_from_low_value(
    grid_countries,
    grid_luc_2019,
    gf,
    inds,
)

In [112]:
t0 = time.time()
tmp = grid_high.data.to_numpy()

t1 = sf.get_time_elapsed(t0)
print(f"array done at {t1} seconds")

tmp = tmp[w_y]
t2 = sf.get_time_elapsed(t0)
print(f"row extraction complete at {t2} seconds")

tmp = tmp[:, w_x]
t3 = sf.get_time_elapsed(t0)
print(f"column extraction complete at {t3} seconds")

array done at 0.0 seconds
row extraction complete at 26.24 seconds
column extraction complete at 35.14 seconds


(array([    0,     0,     0, ..., 43199, 43199, 43199]),
 array([  481,   481,   481, ..., 17280, 17280, 17280]))